In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [28]:
train = pd.read_csv('./input/ch01-titanic/titanic/train.csv')
test = pd.read_csv('./input/ch01-titanic/titanic/test.csv')

In [29]:
train_x = train.drop(["Survived"], axis=1)
train_y = train["Survived"]
test_x = test.copy()

In [30]:
from sklearn.preprocessing import LabelEncoder

In [31]:
train_x = train_x.drop(['PassengerId','Name','Ticket','Cabin'], axis=1)
test_x = test_x.drop(['PassengerId','Name','Ticket','Cabin'], axis=1)

In [32]:
train_x

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...
886,2,male,27.0,0,0,13.0000,S
887,1,female,19.0,0,0,30.0000,S
888,3,female,NaN,1,2,23.4500,S
889,1,male,26.0,0,0,30.0000,C


In [33]:
for c in ['Sex', 'Embarked']:
    le = LabelEncoder()
    le.fit(train_x[c].fillna('NA'))
    
    train_x[c] = le.transform(train_x[c].fillna('NA'))
    test_x[c] = le.transform(test_x[c].fillna('NA'))

In [34]:
from xgboost import XGBClassifier

In [35]:
model = XGBClassifier(n_estimators=20, random_state=71)
model.fit(train_x, train_y)

pred = model.predict_proba(test_x)[:,1]

pred_label = np.where(pred > 0.5, 1, 0)

submission = pd.DataFrame({'PassengerId':test['PassengerId'], 'Survived':pred_label})
submission.to_csv('submission_first.csv', index=False)

In [36]:
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import KFold

In [37]:
scores_accuracy = []
scores_logloss = []

kf = KFold(n_splits=4, shuffle=True, random_state=71)
for tr_idx, va_idx in kf.split(train_x):
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
    
    model = XGBClassifier(n_estimators=20, random_state=71)
    model.fit(tr_x,tr_y)
    
    va_pred = model.predict_proba(va_x)[:,1]
    
    logloss = log_loss(va_y, va_pred)
    accuracy = accuracy_score(va_y, va_pred>0.5)
    
    scores_logloss.append(logloss)
    scores_accuracy.append(accuracy)
    
loggloss = np.mean(scores_logloss)
accuracy = np.mean(scores_accuracy)
print(f'logloss: {logloss:.4f}, accuracy: {accuracy:.4f}')

logloss: 0.4285, accuracy: 0.8148
